In [ ]:
# Tell the notebook where to find the pipeline functions
import sys
sys.path.append('/lab-share/Neuro-Cohen-e2/Public/notebooks/gmiller/Lesion_Pipeline/dependencies/')

In [ ]:
import os
import shutil
from glob import glob 
import GNMPipeline as p

# Add your inputs

In [ ]:
input_dir="output_easyreg_test/"  #Folder with input files, can use the folder created by Combine script previously
input_type='BIDS' #'BIDS' or 'Folder'
output_dir="output_easyreg_test/"

subjects_to_skip=None #['c002', 'c003'] 

mni_software='ANTsQuick' #'EasyReg', 'ANTs', 'ANTsQuick'

lesion_keyword=None #i.e. 'segmentation', or None if you do not have any 

IMAGE_TYPES = ['t1', 'T2', 'FLAIR'] #case sensitive, change to match what you used e.g. t1, t1w, TI 
Reg_target_1='t1' #your ideal registration target, case sensitive
Reg_target_2='T2'

# Make dictionary of subjects and sessions

In [ ]:
subject_sessions=p.create_input_dict(input_dir, subjects_to_skip=subjects_to_skip, input_type=input_type)

# Register Co-registration target to MNI, then bring lesion mask and co-registered images into the same space

In [ ]:
mni_reg_dict={}
for subject, sessions in subject_sessions.items():
    reg_target=None
    subject_folder = os.path.join(output_dir, subject) 
    for session in sessions:
        print(f'*** Processing {subject}: {session} ***')
        session_folder=os.path.join(subject_folder, session)
        
        reg_target=p.set_registration_target(glob(f'{session_folder}/COREGISTERED/*.nii*'), Reg_target_1, Reg_target_2)
        print("Will register", reg_target, "to MNI")
        
        if '1' in reg_target:
            target_image=f'/lab-share/Neuro-Cohen-e2/Public/notebooks/gmiller/Pipeline/0_pipeline_scripts/mni_icbm152_t1_tal_nlin_asym_09c_brain.nii.gz'
            #target_brain_seg= #providing this to EasyReg will save you time
        elif '2' in reg_target:
            target_image=f'/lab-share/Neuro-Cohen-e2/Public/notebooks/gmiller/Pipeline/0_pipeline_scripts/mni_icbm152_t2_tal_nlin_asym_09c_brain.nii.gz'
            #target_brain_seg=
        

        if not os.path.exists(f'{session_folder}/MNI_SPACE'):
            os.makedirs(f'{session_folder}/MNI_SPACE')
            
        source_file=f'{session_folder}/COREGISTERED/{subject}_{session}_{reg_target}.nii.gz'
        
        #look for lesion mask
        if lesion_keyword:
            lesion_mask_files=glob(f'{session_folder}/COREGISTERED/*{lesion_keyword}*nii*') #need to decide where we want these to be found and moved to 
            if lesion_mask_files:
                lesion_mask = lesion_mask_files[0]
            if len(lesion_mask_files) > 1:
                print(f"Warning: Multiple lesion mask files found. Using the first one: {lesion_mask}")
        else:
            lesion_mask = None


        #look for other brains to warp
        other_brains=glob(f'{session_folder}/COREGISTERED/*space-{reg_target}.nii.gz') 
  
        if other_brains:
            print("Found", len(other_brains), "other brains to warp to MNI space also")
        else:
            print("Found no other brains to warp to MNI space")
        
        print(f'Registering the {reg_target} to MNI using {mni_software}') 
        if mni_software == 'EasyReg':
            command=p.easy_reg(f'{session_folder}/MNI_SPACE', source_file, target_image, 'space-MNI', target_brain_seg=target_brain_seg, source_brain_seg=None,
                               lesion_mask=lesion_mask, other_brains=other_brains, synthseg_robust=False)
        elif mni_software == 'ANTs':
            command=p.ants_mni(f'{session_folder}/MNI_SPACE', source_file, target_image, lesion_mask, other_brains, 
                               transform='s', histogram_matching=False, quick=False)
        elif mni_software == 'ANTsQuick':
            command=p.ants_mni(f'{session_folder}/MNI_SPACE', source_file, target_image, lesion_mask, other_brains, 
                               transform='s', histogram_matching=False, quick=True)
        else:
            print("Please set mni_software to 'EasyReg', 'ANTs', or 'ANTsQuick'")
    
            
        job_name = f"MNI-register_{subject}_{session}_{reg_target}"
        job_id=p.submit_slurm_job(job_name, command)
        mni_reg_dict[(subject,session)] = job_id
        

if len(mni_reg_dict) > 0:
    print('You have', len(mni_reg_dict), 'MNI Registration Jobs submitted to SLURM; subject and job IDs are stored in the co_reg_dict')
    print('You can type "squeue -u $USER" into your terminal  or "!squeue -u $USER" in a cell to track SLURM job progress')
    print('You can check the output file matching the jobid in co_reg_dict to see code outputs and any errors')
                                        

# Check your submitted jobs

In [ ]:
print("These are the jobs that should have been submitted:")
print(mni_reg_dict)

In [ ]:
!squeue -u $USER 